# Opening Lead

This is a tutorial for how to use the opening lead engine.

In [6]:
# %pip install --upgrade tensorflow[and-cuda]
# %pip install -q huggingface_hub

In [7]:
import os
os.chdir('..')

from nn.models import Models
from bots import BotLead
from sample import Sample
import conf

In [8]:
import huggingface_hub
import os
lead_id="AIBridgeEngine/Ben-3B-Lead-v0.1"
bidding_id="AIBridgeEngine/Ben-3B-Bidding-v0.1"
biddinginfo_id="AIBridgeEngine/Ben-3B-Biddinginformation-v0.1"
singledummy_id="AIBridgeEngine/Ben-3B-Singledummy-v0.1"
folder="hfmodels"
huggingface_hub.snapshot_download(repo_id=lead_id, local_dir=folder)
huggingface_hub.snapshot_download(repo_id=bidding_id, local_dir=folder)
huggingface_hub.snapshot_download(repo_id=biddinginfo_id, local_dir=folder)
huggingface_hub.snapshot_download(repo_id=singledummy_id, local_dir=folder)

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 14.69it/s]


'D:\\github\\ben\\hfmodels'

In [9]:
from nn.leader import Leader
from nn.bid_info import BidInfo
from nn.bidder import Bidder
from nn.lead_singledummy import LeadSingleDummy
lead = Leader(f'{folder}/lead-1000000')
biddinginfo = BidInfo(f'{folder}/binfo-100000')
bidder = Bidder("", f'{folder}/bidding-100000')
sd_model = LeadSingleDummy(f'{folder}/lr3-1000000')
models = Models(bidder, biddinginfo, lead, sd_model, None, 0.1, 0.05, 1.0, 0.1, False, -1, -1)
sampler = Sample(0.01, 0.1, 0.03, 0.05, 64, 5000, 5000, 128, 100, False)

INFO:tensorflow:Restoring parameters from hfmodels/lead-1000000
INFO:tensorflow:Restoring parameters from hfmodels/binfo-100000
INFO:tensorflow:Restoring parameters from hfmodels/bidding-100000
INFO:tensorflow:Restoring parameters from hfmodels/lr3-1000000


In [10]:
# both vulnerable. you are sitting North as dealer and you hold
hand = 'T54.QT87.A98.943'

# the auction goes:
auction = ["1C","1D","1S","2D","X","PASS","2S","PASS","3C","PASS","3N","PASS","4S",'PASS', 'PASS', 'PASS']

# what to lead?

lead_bot = BotLead([True, True], hand, models, models.lead_threshold, sampler, False)
lead = lead_bot.find_opening_lead(auction)

In [11]:
lead.card

H7

seems like the engine chose to lead the ace of diamonds

the other options it considered were: a small spade and a small club

In [12]:
lead.to_dict()['candidates']

[{'card': 'Hx',
  'insta_score': 0.6122,
  'expected_tricks': 9.98,
  'p_make_contract': 0.65},
 {'card': 'DA',
  'insta_score': 0.1587,
  'expected_tricks': 9.71,
  'p_make_contract': 0.59},
 {'card': 'HT',
  'insta_score': 0.0649,
  'expected_tricks': 9.9,
  'p_make_contract': 0.63}]

in the above output:
- `insta_score` reflects the preference of the neural network
- `expected_tricks` how many tricks declarer is expected to take on this lead
- `p_make_contract` is the probability of the contract making on this lead

the probability of making and the expected tricks are computed on samples which are consistent with the auction. the samples are estimated single dummy using a neural network (more on this in another tutorial). we could also solve the samples double dummy, but that would be quite a bit slower.

In [13]:
# each row is one sample board
# the hands are in the order: LHO, Partner, RHO. Your cards are not shown as they are fixed/

lead.samples

['Axx.KJ9x.K.AJxxx Q98.Ax.Txxxxx.KQ KJxx.xxx.QJx.T8x 0.08078',
 '9xx.AKJx.T.AKJTx A8x.9x.KQxxxx.Qx KQJx.xxx.Jxx.8xx 0.07396',
 '98x.A9xx.K.AKT8x Kxx.Kxx.QJTxx.Qx AQJx.Jx.xxxx.Jxx 0.06952',
 'Axx.AJ.Tx.AK8xxx 98x.Kxxx.KQxxx.x KQJx.9xx.Jxx.QJT 0.06678',
 'Ax.AJxx.xx.AQ8xx KJ9.Kxx.Jxxxx.Jx Q8xxx.9x.KQT.KTx 0.06346',
 'Axx.AKJx.x.KQ8xx K8x.9xxx.KQxxx.x QJ9x.x.JTxx.AJTx 0.05800',
 'Axx.AK9x.x.KQT8x K9x.xx.KJTxx.Axx QJ8x.Jxx.Qxxx.Jx 0.05644',
 'AQ9.K9xx.x.KQJ8x Kx.AJxx.QTxxx.Tx J8xxx.x.KJxx.Axx 0.05347',
 'AJ8.KJxx.x.AK8xx K9x.xxx.KQTxx.Tx Qxxx.A9.Jxxx.QJx 0.05057',
 'A8x.KJxx.x.AKxxx Qxx.A9xx.KQxxx.x KJ9x.x.JTxx.QJT8 0.03821',
 'A8x.Kxxx..AQJTxx K9x.xx.QJTxxx.K8 QJxx.AJ9.Kxxx.xx 0.03808',
 'AKx.xxx..AQJ8xxx J9x.A9x.KQJxxx.x Q8xx.KJx.Txxx.KT 0.03717',
 'AJx.AK9x.x.AJxxx Q8x.xx.KJxxxx.KT K9xx.Jxx.QTx.Q8x 0.03715',
 'K9x.K9xx.x.AKJxx xxx.Axxx.KJxxxx. AQJ8.J.QTx.QT8xx 0.03651',
 'KJx.AK9x.x.QJ8xx 98.Jxxx.Qxxxx.AK AQxxx.x.KJTx.Txx 0.03487',
 'A8x.AK9x.J.ATxxx K9.Jxxx.QTxxx.QJ QJxxx.x.Kxxx.K8x 0.